<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict

def optimize_multigroup_allocation(project_assignments, group_capacities):
    """
    Optimizes point allocation across multiple groups for shared projects.
    
    Args:
        project_assignments (dict): Points assigned to each group for each project:
            {project_name: {
                'deadline_months': int,
                'group_points': {group_name: points_to_complete}
            }}
        group_capacities (dict): Points each group can allocate per month:
            {group_name: int}
    
    Returns:
        dict: Allocation history for each group
        dict: Project completion status
    """
    # Initialize tracking for each group-project combination
    group_project_status = defaultdict(dict)
    project_total_points = {}
    
    for proj_name, proj_info in project_assignments.items():
        total_points = 0
        for group_name, points_needed in proj_info['group_points'].items():
            group_project_status[group_name][proj_name] = {
                'points_needed': points_needed,
                'points_remaining': points_needed,
                'deadline_months': proj_info['deadline_months'],
                'completed': False,
                'completion_month': None
            }
            total_points += points_needed
        project_total_points[proj_name] = total_points
    
    # Initialize allocation history
    allocation_history = {group: defaultdict(dict) for group in group_capacities}
    
    # Track overall project completion
    project_completion_status = {
        proj: {
            'total_points': project_total_points[proj],
            'deadline_months': project_assignments[proj]['deadline_months'],
            'completed': False,
            'completion_month': None
        } for proj in project_assignments
    }
    
    month = 1
    
    print("=== Multi-Group Project Allocation Simulation ===\n")
    print("Project Breakdown:")
    for proj_name, proj_info in project_assignments.items():
        print(f"\n{proj_name} (Deadline: Month {proj_info['deadline_months']}):")
        for group, points in proj_info['group_points'].items():
            print(f"  - {group}: {points} points")
        print(f"  Total: {project_total_points[proj_name]} points")
    
    while any(not status['completed'] for group_tasks in group_project_status.values() 
             for status in group_tasks.values()):
        print(f"\n\n--- Month {month} ---")
        
        # For each group, calculate urgency of their assigned tasks
        monthly_allocations = defaultdict(dict)
        
        for group_name, capacity in group_capacities.items():
            group_tasks = group_project_status[group_name]
            
            # Calculate urgency for each project this group needs to work on
            project_urgencies = []
            for proj_name, status in group_tasks.items():
                if status['completed']:
                    continue
                
                months_left = status['deadline_months'] - (month - 1)
                if months_left <= 0:
                    urgency = float('inf')
                    print(f"🚨 URGENT: {group_name} is at/past deadline for '{proj_name}'!")
                else:
                    urgency = status['points_remaining'] / months_left
                
                project_urgencies.append((proj_name, urgency, status['points_remaining']))
            
            # Sort by urgency
            project_urgencies.sort(key=lambda x: x[1], reverse=True)
            
            # Allocate points
            remaining_capacity = capacity
            print(f"\n{group_name} (capacity: {capacity} points):")
            
            if not project_urgencies:
                print("  No active projects")
                continue
            
            print(f"  Priorities: {[f'{p[0]} (urgency: {p[1]:.2f})' for p in project_urgencies[:3]]}")
            
            for proj_name, urgency, points_needed in project_urgencies:
                if remaining_capacity <= 0:
                    break
                
                points_to_allocate = min(remaining_capacity, points_needed)
                monthly_allocations[group_name][proj_name] = points_to_allocate
                remaining_capacity -= points_to_allocate
                
                # Update status
                group_tasks[proj_name]['points_remaining'] -= points_to_allocate
                
                print(f"  → Allocated {points_to_allocate} points to '{proj_name}'")
                
                if group_tasks[proj_name]['points_remaining'] == 0:
                    group_tasks[proj_name]['completed'] = True
                    group_tasks[proj_name]['completion_month'] = month
                    print(f"    ✓ {group_name} completed their portion of '{proj_name}'!")
            
            if remaining_capacity > 0:
                print(f"  ⚠️  Unused capacity: {remaining_capacity} points")
        
        # Record allocations
        for group_name in group_capacities:
            for proj_name in project_assignments:
                allocation_history[group_name][month][proj_name] = (
                    monthly_allocations[group_name].get(proj_name, 0)
                )
        
        # Check for overall project completion
        for proj_name in project_assignments:
            if not project_completion_status[proj_name]['completed']:
                all_groups_done = all(
                    group_project_status[group].get(proj_name, {}).get('completed', True)
                    for group in project_assignments[proj_name]['group_points']
                )
                if all_groups_done:
                    project_completion_status[proj_name]['completed'] = True
                    project_completion_status[proj_name]['completion_month'] = month
                    print(f"\n🎉 PROJECT '{proj_name}' FULLY COMPLETED! 🎉")
        
        month += 1
        
        # Safety check
        if month > 100:
            print("\nWarning: Simulation stopped after 100 months")
            break
    
    print("\n=== Simulation Complete ===")
    
    return allocation_history, group_project_status, project_completion_status


def create_allocation_dataframes(allocation_history, project_assignments):
    """
    Convert allocation history to pandas DataFrames for each group.
    """
    dataframes = {}
    
    for group_name, history in allocation_history.items():
        months = sorted(history.keys())
        project_names = sorted(project_assignments.keys())
        
        allocation_matrix = []
        for proj in project_names:
            row = [history.get(month, {}).get(proj, 0) for month in months]
            allocation_matrix.append(row)
        
        df = pd.DataFrame(
            allocation_matrix,
            index=project_names,
            columns=[f"Month {m}" for m in months]
        )
        
        dataframes[group_name] = df
    
    return dataframes


def create_assignment_summary(project_assignments):
    """
    Create a summary table of project assignments.
    """
    print("\n=== PROJECT ASSIGNMENT SUMMARY ===")
    
    # Get all groups
    all_groups = set()
    for proj_info in project_assignments.values():
        all_groups.update(proj_info['group_points'].keys())
    all_groups = sorted(all_groups)
    
    # Build matrix
    matrix = []
    project_names = []
    deadlines = []
    
    for proj_name, proj_info in sorted(project_assignments.items()):
        project_names.append(proj_name)
        deadlines.append(proj_info['deadline_months'])
        row = []
        for group in all_groups:
            points = proj_info['group_points'].get(group, 0)
            row.append(points)
        matrix.append(row)
    
    df = pd.DataFrame(matrix, index=project_names, columns=all_groups)
    df['Total'] = df.sum(axis=1)
    df['Deadline'] = deadlines
    
    print("\nPoints assigned to each group by project:")
    print(df)
    
    return df


def create_detailed_report(allocation_dfs, group_project_status, project_completion_status, 
                          group_capacities, project_assignments):
    """
    Create a detailed report of the allocation results.
    """
    print("\n\n=== DETAILED ALLOCATION REPORT ===")
    
    # Project completion summary
    print("\nProject Completion Summary:")
    for proj_name, status in project_completion_status.items():
        deadline = status['deadline_months']
        if status['completed']:
            completion_month = status['completion_month']
            on_time = completion_month <= deadline
            status_str = "✅ ON TIME" if on_time else "⚠️  LATE"
            print(f"  {proj_name}: Completed Month {completion_month} (Deadline: Month {deadline}) {status_str}")
        else:
            print(f"  {proj_name}: ❌ NOT COMPLETED (Deadline: Month {deadline})")
    
    # Group performance summary
    print("\n\nGroup Performance Summary:")
    for group_name, df in allocation_dfs.items():
        capacity = group_capacities[group_name]
        group_tasks = group_project_status[group_name]
        
        # Calculate metrics
        total_points_allocated = df.sum().sum()
        months_active = len([m for m in df.columns if df[m].sum() > 0])
        
        if months_active > 0:
            avg_utilization = df.sum(axis=0)[:months_active].mean() / capacity
            max_utilization = df.sum(axis=0).max() / capacity
        else:
            avg_utilization = 0
            max_utilization = 0
        
        # Count completed tasks
        tasks_completed = sum(1 for task in group_tasks.values() if task['completed'])
        tasks_total = len(group_tasks)
        
        print(f"\n{group_name} (Capacity: {capacity} points/month):")
        print(f"  Tasks: {tasks_completed}/{tasks_total} completed")
        print(f"  Total points delivered: {int(total_points_allocated)}")
        print(f"  Active months: {months_active}")
        print(f"  Average utilization: {avg_utilization:.1%}")
        print(f"  Peak utilization: {max_utilization:.1%}")
        
        # Show incomplete tasks
        incomplete = [proj for proj, status in group_tasks.items() if not status['completed']]
        if incomplete:
            print(f"  ⚠️  Incomplete tasks: {', '.join(incomplete)}")


# Example usage
if __name__ == "__main__":
    # Define projects with work distributed among groups
    project_assignments = {
        'Project Alpha': {
            'deadline_months': 4,
            'group_points': {
                'Group A': 20,  # Group A must complete 20 points
                'Group C': 10   # Group C must complete 10 points
            }
        },
        'Project Beta': {
            'deadline_months': 6,
            'group_points': {
                'Group A': 30,  # Group A must complete 30 points
                'Group B': 20   # Group B must complete 20 points
            }
        },
        'Project Gamma': {
            'deadline_months': 3,
            'group_points': {
                'Group A': 5,   # Group A must complete 5 points
                'Group B': 15   # Group B must complete 15 points
            }
        },
        'Project Delta': {
            'deadline_months': 5,
            'group_points': {
                'Group B': 25,  # Group B must complete 25 points
                'Group C': 15   # Group C must complete 15 points
            }
        },
        'Project Epsilon': {
            'deadline_months': 2,
            'group_points': {
                'Group B': 8,   # Group B must complete 8 points
                'Group C': 7    # Group C must complete 7 points
            }
        }
    }
    
    # Define group capacities (points per month)
    group_capacities = {
        'Group A': 15,
        'Group B': 10,
        'Group C': 8
    }
    
    # Show assignment summary
    assignment_df = create_assignment_summary(project_assignments)
    
    # Run the optimization
    allocation_history, group_project_status, project_completion_status = optimize_multigroup_allocation(
        project_assignments, group_capacities
    )
    
    # Create DataFrames for visualization
    allocation_dfs = create_allocation_dataframes(allocation_history, project_assignments)
    
    # Print allocation tables
    print("\n\n=== MONTHLY ALLOCATION TABLES ===")
    for group_name, df in allocation_dfs.items():
        print(f"\n{group_name} Monthly Allocations:")
        print(df)
        monthly_totals = df.sum(axis=0).values
        print(f"Monthly totals: {[int(x) for x in monthly_totals]}")
        print(f"Utilization: {[f'{int(used)}/{group_capacities[group_name]}' for used in monthly_totals]}")
    
    # Create detailed report
    create_detailed_report(allocation_dfs, group_project_status, project_completion_status,
                          group_capacities, project_assignments)
    
    # Show combined project progress
    print("\n\n=== COMBINED PROJECT PROGRESS ===")
    combined_df = pd.DataFrame()
    for group_name, df in allocation_dfs.items():
        if combined_df.empty:
            combined_df = df.copy()
        else:
            combined_df = combined_df.add(df, fill_value=0)
    
    print("Total points allocated to each project per month (all groups combined):")
    print(combined_df.astype(int))
    
    # Calculate cumulative progress
    print("\nCumulative progress by project:")
    cumulative = combined_df.cumsum(axis=1)
    for proj_name in project_assignments:
        total_needed = project_assignments[proj_name]['group_points']
        total_points = sum(total_needed.values())
        progress = cumulative.loc[proj_name]
        completion_month = None
        for month, points in progress.items():
            if points >= total_points:
                completion_month = month
                break
        print(f"  {proj_name}: {total_points} points needed, completed in {completion_month}")


=== PROJECT ASSIGNMENT SUMMARY ===

Points assigned to each group by project:
                 Group A  Group B  Group C  Total  Deadline
Project Alpha         20        0       10     30         4
Project Beta          30       20        0     50         6
Project Delta          0       25       15     40         5
Project Epsilon        0        8        7     15         2
Project Gamma          5       15        0     20         3
=== Multi-Group Project Allocation Simulation ===

Project Breakdown:

Project Alpha (Deadline: Month 4):
  - Group A: 20 points
  - Group C: 10 points
  Total: 30 points

Project Beta (Deadline: Month 6):
  - Group A: 30 points
  - Group B: 20 points
  Total: 50 points

Project Gamma (Deadline: Month 3):
  - Group A: 5 points
  - Group B: 15 points
  Total: 20 points

Project Delta (Deadline: Month 5):
  - Group B: 25 points
  - Group C: 15 points
  Total: 40 points

Project Epsilon (Deadline: Month 2):
  - Group B: 8 points
  - Group C: 7 points
  Total

In [5]:
cumulative

,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7
Project Alpha,15,15,23,30,30,30,30
Project Beta,0,15,25,35,35,42,50
Project Delta,1,11,21,27,37,40,40
Project Epsilon,7,15,15,15,15,15,15
Project Gamma,10,10,15,20,20,20,20


In [6]:
combined_df

,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7
Project Alpha,15,0,8,7,0,0,0
Project Beta,0,15,10,10,0,7,8
Project Delta,1,10,10,6,10,3,0
Project Epsilon,7,8,0,0,0,0,0
Project Gamma,10,0,5,5,0,0,0


In [10]:
allocation_dfs

{'Group A':                  Month 1  Month 2  Month 3  Month 4  Month 5  Month 6  Month 7
 Project Alpha         15        0        0        5        0        0        0
 Project Beta           0       15       10        5        0        0        0
 Project Delta          0        0        0        0        0        0        0
 Project Epsilon        0        0        0        0        0        0        0
 Project Gamma          0        0        5        0        0        0        0,
 'Group B':                  Month 1  Month 2  Month 3  Month 4  Month 5  Month 6  Month 7
 Project Alpha          0        0        0        0        0        0        0
 Project Beta           0        0        0        5        0        7        8
 Project Delta          0        2       10        0       10        3        0
 Project Epsilon        0        8        0        0        0        0        0
 Project Gamma         10        0        0        5        0        0        0,
 'Group C':     